In [ ]:
pip install pandas

: 

In [ ]:
pip install mediapipe 

In [ ]:
pip install tensorflow

In [ ]:
pip install opencv-python tensorflow keras numpy

In [ ]:
pip install matplotlib seaborn


In [ ]:
pip install joblib


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load dataset
def load_data():
    # Example: Generate synthetic data
    X = np.random.rand(1000, 10)  # 1000 samples, 10 features
    y = np.random.randint(0, 2, size=(1000, 1))  # Binary target
    return X, y

# Preprocess data
def preprocess_data(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

# Build model
def build_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    return model

# Compile model
def compile_model(model):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
def train_model(model, X_train, y_train, epochs=10, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Main function
if __name__ == "__main__":   # ✅ fixed
    X, y = load_data()
    X_train, X_test, y_train, y_test = preprocess_data(X, y)
    model = build_model(X_train.shape[1])
    compile_model(model)
    train_model(model, X_train, y_train)

    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_accuracy:.4f}")


In [ ]:

pip install opencv-contrib-python


In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os
from collections import Counter

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

gestures = {
    0: "Hello",
    1: "Yes", 
    2: "No",
    3: "Thank You",
    4: "I Love You"
}

def normalize_landmarks(landmarks):
    """Normalize landmarks relative to wrist position"""
    landmarks = np.array(landmarks).reshape(-1, 2)
    wrist = landmarks[0]
    normalized = landmarks - wrist
    if len(landmarks) > 12:
        hand_size = np.linalg.norm(landmarks[12] - wrist)
        if hand_size > 0:
            normalized = normalized / hand_size
    return normalized.flatten()

def extract_features(landmarks):
    """Extract more robust features from landmarks"""
    landmarks = np.array(landmarks).reshape(-1, 2)
    features = []
    normalized = normalize_landmarks(landmarks.flatten())
    features.extend(normalized)

    # Distances between key points
    key_points = [0, 4, 8, 12, 16, 20]
    for i in range(len(key_points)):
        for j in range(i+1, len(key_points)):
            if key_points[i] < len(landmarks) and key_points[j] < len(landmarks):
                dist = np.linalg.norm(landmarks[key_points[i]] - landmarks[key_points[j]])
                features.append(dist)



In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
import csv
from collections import defaultdict

# ========== CONFIG ==========
GESTURE_LABELS = {
    0: "Hello",
    1: "Yes",
    2: "No",
    3: "Thank You",
    4: "I Love You",
    5: "Stop",
    6: "Help",
    7: "Wait",
    8: "Bye",
    9: "Good Morning",
    10: "Good Night",
    11: "Sorry",
    12: "OK",
    13: "Peace",
    14: "Rock",
    15: "Thumbs Up",
    16: "Thumbs Down",
    17: "Call Me",
    18: "Hungry",
    19: "Thirsty",
    20: "Pain",
    21: "Emergency",
    22: "Washroom",
    23: "Come Here",
    24: "Go Away",
    25: "Love",
    26: "Angry",
    27: "Confused",
    28: "Relax",
    29: "Super"
}

SAMPLES_PER_LABEL = 40      # how many samples to collect per gesture
COUNTDOWN_SECS = 3         # countdown before capturing each batch
CAPTURE_INTERVAL = 0.4     # seconds between each sample capture
SAVE_IMAGES = True         # save camera images of each sample (helpful for debugging)
OUTPUT_DIR = "gesture_dataset"
IMAGES_DIR = os.path.join(OUTPUT_DIR, "images")
LANDMARKS_FILE = os.path.join(OUTPUT_DIR, "landmarks.npz")
META_CSV = os.path.join(OUTPUT_DIR, "metadata.csv")
# ============================

os.makedirs(OUTPUT_DIR, exist_ok=True)
if SAVE_IMAGES:
    os.makedirs(IMAGES_DIR, exist_ok=True)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def normalize_landmarks_flat(landmarks_flat):
    """
    landmarks_flat: list or array [x0,y0,x1,y1,...]
    returns normalized flattened vector: coordinates shifted by wrist and scaled by hand size
    """
    arr = np.array(landmarks_flat).reshape(-1, 2)
    wrist = arr[0].copy()
    arr = arr - wrist
    if arr.shape[0] > 12:
        hand_size = np.linalg.norm(arr[12])  # after subtracting wrist, index 12 is distance vector
        if hand_size > 1e-6:
            arr = arr / hand_size
    return arr.flatten()

def landmarks_to_flat(landmarks):
    return [lm.x for lm in landmarks] + [lm.y for lm in landmarks]  # first all x then all y? (we'll keep x,y pair order)
    # NOTE: above line actually concatenates xs then ys — we'll prefer x,y pairs:
def landmarks_to_flat_xy(landmarks):
    flat = []
    for lm in landmarks:
        flat.extend([lm.x, lm.y])
    return flat

def draw_overlay(frame, title, subtitle, progress_text):
    h, w = frame.shape[:2]
    # translucent background
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (w, 75), (0, 0, 0), -1)
    alpha = 0.5
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
    cv2.putText(frame, title, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
    cv2.putText(frame, subtitle, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 1)
    if progress_text:
        cv2.putText(frame, progress_text, (w - 10 - 400, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Could not open camera. Check your webcam.")
        return

    all_landmarks = []   # list of normalized flattened vectors
    all_labels = []      # integer labels
    metadata = []        # rows: [label, label_name, timestamp, image_path (optional)]

    # For quick progress display
    counts = defaultdict(int)

    with mp_hands.Hands(
        max_num_hands=1,
        min_detection_confidence=0.6,
        min_tracking_confidence=0.5
    ) as hands:

        for label_id in sorted(GESTURE_LABELS.keys()):
            label_name = GESTURE_LABELS[label_id]
            print(f"\n--- Preparing to collect for [{label_id}] {label_name} ---")
            # Wait for user to be ready
            ready = False
            while not ready:
                ret, frame = cap.read()
                if not ret:
                    print("Error reading frame.")
                    return
                frame = cv2.flip(frame, 1)
                draw_overlay(frame,
                             f"Prepare for: {label_id} - {label_name}",
                             f"Hold the gesture. Countdown starts when you press SPACE. Press 's' to skip label.",
                             f"Collected: {counts[label_id]}/{SAMPLES_PER_LABEL}")
                cv2.imshow("Capture Gestures", frame)
                k = cv2.waitKey(1) & 0xFF
                if k == ord(' '):   # SPACE starts countdown
                    ready = True
                elif k == ord('s'): # skip this label
                    print(f"Skipped label {label_id}")
                    break
                elif k == ord('q'):
                    print("Quitting early by user.")
                    cap.release()
                    cv2.destroyAllWindows()
                    # Save whatever we have
                    save_dataset(all_landmarks, all_labels, metadata)
                    return

            if not ready:
                continue

            # countdown
            for c in range(COUNTDOWN_SECS, 0, -1):
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.flip(frame, 1)
                draw_overlay(frame,
                             f"Get ready: {label_name}",
                             f"Capturing in... {c}",
                             f"Collected: {counts[label_id]}/{SAMPLES_PER_LABEL}")
                cv2.imshow("Capture Gestures", frame)
                cv2.waitKey(1000)

            print(f"Start capturing {SAMPLES_PER_LABEL} samples for '{label_name}'")

            # capture loop
            captured = 0
            last_capture_time = 0
            while captured < SAMPLES_PER_LABEL:
                ret, frame = cap.read()
                if not ret:
                    print("Frame read error, trying again...")
                    continue
                frame = cv2.flip(frame, 1)
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(rgb)

                # draw landmarks if found
                if results and results.multi_hand_landmarks:
                    for hl in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hl, mp_hands.HAND_CONNECTIONS)

                now = time.time()
                if now - last_capture_time >= CAPTURE_INTERVAL:
                    # if a hand is detected, capture; otherwise wait
                    if results and results.multi_hand_landmarks:
                        hand_landmarks = results.multi_hand_landmarks[0].landmark
                        flat = landmarks_to_flat_xy(hand_landmarks)
                        norm = normalize_landmarks_flat(flat)
                        all_landmarks.append(norm)
                        all_labels.append(label_id)
                        ts = time.strftime("%Y%m%d-%H%M%S")
                        img_path = ""
                        if SAVE_IMAGES:
                            img_name = f"label_{label_id}_{label_name.replace(' ', '_')}_{counts[label_id]:03d}_{ts}.jpg"
                            img_path = os.path.join(IMAGES_DIR, img_name)
                            cv2.imwrite(img_path, frame)
                        metadata.append([label_id, label_name, time.time(), img_path])
                        captured += 1
                        counts[label_id] += 1
                        last_capture_time = now
                        print(f"Captured {captured}/{SAMPLES_PER_LABEL} for {label_name}")

                draw_overlay(frame,
                             f"Collecting: {label_id} - {label_name}",
                             f"Samples: {captured}/{SAMPLES_PER_LABEL}  (Press 'q' to stop)",
                             f"Total saved so far: {sum(counts.values())}")
                cv2.imshow("Capture Gestures", frame)
                k = cv2.waitKey(1) & 0xFF
                if k == ord('q'):
                    print("User requested early quit.")
                    cap.release()
                    cv2.destroyAllWindows()
                    save_dataset(all_landmarks, all_labels, metadata)
                    return

            print(f"Completed label {label_id} ({label_name}). Moving to next.")

    # finished all labels
    cap.release()
    cv2.destroyAllWindows()
    save_dataset(all_landmarks, all_labels, metadata)
    print("All done. Dataset saved.")

def save_dataset(landmarks_list, labels_list, metadata_rows):
    if len(landmarks_list) == 0:
        print("No samples to save.")
        return

    # pad to same length if needed and stack
    max_len = max(len(x) for x in landmarks_list)
    X = np.array([np.pad(x, (0, max_len - len(x)), 'constant') for x in landmarks_list])
    y = np.array(labels_list)

    # save numpy compressed
    np.savez_compressed(LANDMARKS_FILE, X=X, y=y)
    print(f"Saved landmarks -> {LANDMARKS_FILE}  (samples: {len(y)})")

    # save metadata csv
    header = ["label_id", "label_name", "timestamp", "image_path"]
    with open(META_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for row in metadata_rows:
            writer.writerow(row)
    print(f"Saved metadata -> {META_CSV}")

if __name__ == "__main__":
    main()



--- Preparing to collect for [0] Hello ---
